# TRY 2

In [5]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

import time, re
import textblob
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pconn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
urls = [#common
        (1,'https://www.bestbuy.com/site/apple-iphone-14-128gb-unlocked-midnight/6507555.p?skuId=6507555'),
        (2,'https://www.bestbuy.com/site/samsung-galaxy-s22-ultra-128gb-phantom-black-verizon/6494471.p?skuId=6494471'),
        (3,'https://www.bestbuy.com/site/hp-deskjet-2755e-wireless-inkjet-printer-with-3-months-of-instant-ink-included-with-hp-white/6454282.p?skuId=6454282'),
        (4,'https://www.bestbuy.com/site/jbl-flip-5-portable-bluetooth-speaker-black/6356535.p?skuId=6356535'),
        (5,'https://www.bestbuy.com/site/microsoft-365-family-up-to-6-people-12-month-subscription-activation-required-windows-mac-os-apple-ios-android-digital/6258038.p?skuId=6258038'),
        (6,'https://www.bestbuy.com/site/turbotax-deluxe-2023-federal-e-file-state-mac-os-windows-digital/6560825.p?skuId=6560825'),
        (7,'https://www.bestbuy.com/site/hamilton-beach-classic-4-slice-toaster-with-sure-toast-technology-stainless-steel/6518125.p?skuId=6518125'),
        (8,'https://www.bestbuy.com/site/razor-mx350-dirt-rocket-ebike-w-7-miles-max-operating-range-14-mph-max-speed-small-blue/3698469.p?skuId=3698469'),
        (9,'https://www.bestbuy.com/site/lg-65-class-c3-series-oled-4k-uhd-smart-webos-tv/6535929.p?skuId=6535929'),
        # NO LONGER AVAILABLE ON BESTBUY (10,'https://www.bestbuy.com/site/ge-1-4-cu-ft-mid-size-microwave-black/3180025.p?skuId=3180025',
        (11,'https://www.bestbuy.com/site/apple-iphone-13-5g-128gb-unlocked-midnight/6417788.p?skuId=6417788'),
        (12,'https://www.bestbuy.com/site/hogwarts-legacy-standard-edition-xbox-series-x/6502627.p?skuId=6502627'),
        (13,'https://www.bestbuy.com/site/star-wars-jedi-survivor-standard-edition-playstation-5/6528657.p?skuId=6528657'),
        (14,'https://www.bestbuy.com/site/ninja-woodfire-pizza-oven-8-in-1-outdoor-oven-5-pizza-settings-700f-smoker-woodfire-technology-electric-terracotta-red/6548901.p?skuId=6548901'),
        (15,'https://www.bestbuy.com/site/igloo-imx-24-quart-cooler-gray/6523836.p?skuId=6523836'),
        #BB exclusive
        (40,'https://www.bestbuy.com/site/mcafee-total-protection-5-device-antivirus-internet-security-software-1-year-subscription-android-apple-ios-chrome-mac-os-windows-digital/6517326.p?skuId=6517326'),
        (41,'https://www.bestbuy.com/site/norton-360-deluxe-5-device-antivirus-internet-security-software-vpn-dark-web-monitoring-1-year-subscription-android-mac-os-windows-apple-ios-digital/6346690.p?skuId=6346690'),
        ]

s = Service() #Service(executable_path='D:\CU Boulder - Data Science\Spring 2024\Data Mining\chromedriver.exe')

In [3]:
Global_Reviewer_name_lst = []
Global_Review_title_lst = []
Global_Reviewer_star_rating_lst = []
Global_Review_content_lst = []
Global_Helpfulness_votes_lst = []
Global_review_ID = []
Global_product_site_id = []
Global_product_id = []
Global_product_site_id = []
Global_verified_purchase = []

Global_product_name_lst = []
Global_product_star_rating_lst = []
Global_product_total_reviews = []

In [6]:
pat_review_content = []

for idx,url in enumerate(urls):
    
    driver = webdriver.Chrome(service=s)
    driver.get(url[1])
    driver.maximize_window()
    reviews_button = driver.find_element(By.LINK_TEXT,'See All Customer Reviews')
    reviews_button.click()

    html_product = driver.find_element(By.CLASS_NAME,'row').get_attribute('outerHTML')
    product_soup = BeautifulSoup(html_product,'html.parser')

    product_id = url[0] #idx+1

    site_id = 'BB'+str(product_id)
    # Product name
    product_name = product_soup.find('a',class_='v-fw-regular').text
    # Product star rating
    product_star_rating = product_soup.find('span', class_ ='ugc-c-review-average font-weight-medium order-1').text

    # Total product reviews
    text_product_total_reviews = product_soup.find('span', class_ ='c-reviews order-2').text

    match = re.search(r'\(([\d,]+)\sReviews\)', text_product_total_reviews)
    str_product_total_reviews = match.group(1)
    product_total_reviews = int(str_product_total_reviews.replace(',', ''))

    
    Global_product_name_lst.append(product_name)
    Global_product_star_rating_lst.append(product_star_rating)
    Global_product_total_reviews.append(product_total_reviews)

    # Reviewer_name_lst = []
    # Review_title_lst = []
    # Reviewer_star_rating_lst = []
    # Review_content_lst = []
    # Helpfulness_votes_lst = []

    n = round(product_total_reviews/20)
    review_id = 0

    for j in range(1,n+1):
        
        whole_page = driver.find_element(By.ID,'ugc-ratings-and-reviews')

        # whole_page = driver.find_element(By.CLASS_NAME,'ugc-reviews reviews-content-wrapper v-m-top-xs')
        time.sleep(3) #3)
        #html_review = None
        #html_review = whole_page.find_element(By.CLASS_NAME,'reviews-list').get_attribute('outerHTML')
        html_review = WebDriverWait(driver,timeout=8,poll_frequency=0.1).until(
            EC.visibility_of_element_located((By.CLASS_NAME,'reviews-list'))
        ).get_attribute('outerHTML')
        whole_page_soup = BeautifulSoup(html_review,'html.parser')
        all_reviews = whole_page_soup.find_all('li',class_='review-item')
        all_reviews

        for i in all_reviews:

            verified_purchase = True
            review_id +=1
            review_info = {}

            # Reviewer's name
            reviewer_name = i.find('div', class_='ugc-author v-fw-medium body-copy-lg').text
            # Review title
            review_title = i.find('h4',class_='c-section-title review-title heading-5 v-fw-medium').text         
            # Star rating of every review
            review_star_rating = i.find('p', class_ ='visually-hidden').text[6:7]
            # Content of the review
            review_content = i.find('p', class_='pre-white-space').text
            # Helpfulness votes
            review_helpful_votes = i.find('button',class_='c-button c-button-outline c-button-sm helpfulness-button no-margin-l').text.split('(')[1].split(')')[0]

            # review_info['reviewer_name'] = reviewer_name#i.find('div', class_='ugc-author v-fw-medium body-copy-lg').text
            # review_info['review_title'] = review_title#i.find('h4',class_='c-section-title review-title heading-5 v-fw-medium').text
            # review_info['review_star_rating']=review_star_rating #i.find('h4',class_='c-section-title review-title heading-5 v-fw-medium').text
            # review_info['review_content'] = review_content #i.find('p', class_='pre-white-space').text
            # review_info['review_helpful_votes'] = review_helpful_votes #i.find('button',class_='c-button c-button-outline c-button-sm helpfulness-button no-margin-l').text.split('(')[1].split(')')[0]
            
            # Reviewer_name_lst.append(reviewer_name)
            # Review_title_lst.append(review_title)
            # Reviewer_star_rating_lst.append(review_star_rating)
            # Review_content_lst.append(review_content)
            # Helpfulness_votes_lst.append(review_helpful_votes)
            Global_verified_purchase.append(verified_purchase)
            Global_review_ID.append(review_id)
            Global_product_site_id.append(site_id)
            Global_product_id.append(product_id)
            Global_Reviewer_name_lst.append(reviewer_name)
            Global_Review_title_lst.append(review_title)
            Global_Reviewer_star_rating_lst.append(review_star_rating)
            Global_Review_content_lst.append(review_content)
            Global_Helpfulness_votes_lst.append(review_helpful_votes)
            
            pat_review_content.append(review_info)

        next_page_buton = driver.find_element(By.XPATH,"//a[@data-track='Page next']")
        next_page_buton.click()
    print(f"{url[1]}: done")
    driver.quit()


https://www.bestbuy.com/site/apple-iphone-14-128gb-unlocked-midnight/6507555.p?skuId=6507555done
https://www.bestbuy.com/site/samsung-galaxy-s22-ultra-128gb-phantom-black-verizon/6494471.p?skuId=6494471done
https://www.bestbuy.com/site/hp-deskjet-2755e-wireless-inkjet-printer-with-3-months-of-instant-ink-included-with-hp-white/6454282.p?skuId=6454282done
https://www.bestbuy.com/site/jbl-flip-5-portable-bluetooth-speaker-black/6356535.p?skuId=6356535done
https://www.bestbuy.com/site/microsoft-365-family-up-to-6-people-12-month-subscription-activation-required-windows-mac-os-apple-ios-android-digital/6258038.p?skuId=6258038done
https://www.bestbuy.com/site/turbotax-deluxe-2023-federal-e-file-state-mac-os-windows-digital/6560825.p?skuId=6560825done
https://www.bestbuy.com/site/hamilton-beach-classic-4-slice-toaster-with-sure-toast-technology-stainless-steel/6518125.p?skuId=6518125done
https://www.bestbuy.com/site/razor-mx350-dirt-rocket-ebike-w-7-miles-max-operating-range-14-mph-max-speed

In [ ]:
# len(pd.DataFrame(pat_review_content)['reviewer_name'])

In [ ]:
len(Global_Review_title_lst)

26190

In [ ]:
product_data = {
    'product_title': Global_product_name_lst,  
    'Star_rating': Global_product_star_rating_lst,
    'Total_reviews': Global_product_total_reviews
}

df_product = pd.DataFrame(product_data)
df_product

,product_title,Star_rating,Total_reviews
0,Apple - iPhone 14 128GB (Unlocked) - Midnight,4.9,215
1,Samsung - Galaxy S22 Ultra 128GB - Phantom Bla...,4.6,396
2,HP - DeskJet 2755e Wireless Inkjet Printer wit...,4.1,3510
3,JBL - Flip 5 Portable Bluetooth Speaker - Black,4.8,5953
4,Microsoft - 365 Family (Up to 6 People) (12-Mo...,4.5,10372
5,TurboTax - Deluxe 2023 Federal + E-file & Stat...,4.7,337
6,Hamilton Beach - Classic 4 Slice Toaster with ...,4.5,28
7,Razor - MX350 Dirt Rocket eBike w/7 miles Max ...,4.4,201
8,"LG - 65"" Class C3 Series OLED 4K UHD Smart web...",4.8,1530
9,Apple - iPhone 13 5G 128GB (Unlocked) - Midnight,4.8,1003


In [44]:
df_product_without_duplicates = df_product.drop_duplicates()

df_products = df_product_without_duplicates.reset_index(drop=True)
df_products

,product_title,Star_rating,Total_reviews
0,Apple - iPhone 14 128GB (Unlocked) - Midnight,4.9,215
1,Samsung - Galaxy S22 Ultra 128GB - Phantom Bla...,4.6,396
2,HP - DeskJet 2755e Wireless Inkjet Printer wit...,4.1,3510
3,JBL - Flip 5 Portable Bluetooth Speaker - Black,4.8,5953
4,Microsoft - 365 Family (Up to 6 People) (12-Mo...,4.5,10372
5,TurboTax - Deluxe 2023 Federal + E-file & Stat...,4.7,337
6,Hamilton Beach - Classic 4 Slice Toaster with ...,4.5,28
7,Razor - MX350 Dirt Rocket eBike w/7 miles Max ...,4.4,201
8,"LG - 65"" Class C3 Series OLED 4K UHD Smart web...",4.8,1530
9,Apple - iPhone 13 5G 128GB (Unlocked) - Midnight,4.8,1003


In [7]:
len(Global_Review_title_lst)

28192

In [8]:
review_data = {
    'productID': Global_product_id,
    'prodSiteID': Global_product_site_id,
    'ReviewID': Global_review_ID,
    'review_header':Global_Review_title_lst,
    'reviewer_name':Global_Reviewer_name_lst,
    'review_content':Global_Review_content_lst,
    'review_star_rating':Global_Reviewer_star_rating_lst,
    'review_helpful_votes':Global_Helpfulness_votes_lst,
    'verified_purchase' : Global_verified_purchase
}

df_review1 = pd.DataFrame(review_data)
df_review1

,productID,prodSiteID,ReviewID,review_header,reviewer_name,review_content,review_star_rating,review_helpful_votes,verified_purchase
0,1,BB1,1,Awesome experience all around,AmandaD,"Husband upgraded from a 12 to a 14, loves it. ...",5,0,True
1,1,BB1,2,Great phone,MauraC,It’s a pretty great phone. It has all that I n...,5,0,True
2,1,BB1,3,It’s an iPhone,Peachy,"It’s a unlocked iPhone, I was able to pick up ...",5,0,True
3,1,BB1,4,Hello….Can You Hear Me Now?,PauletteP,Apple products are very good products but of c...,5,0,True
4,1,BB1,5,Great!!,AnthonyC,Awesome product love it so much. I would recom...,5,0,True
...,...,...,...,...,...,...,...,...,...
28187,41,BB41,356,Returned this product,TraceyPortchester,did not have the capacity to safeguard all of ...,1,0,True
28188,41,BB41,357,forget BB,Deede,Received in email but not downloadable. Geek ...,1,0,True
28189,41,BB41,358,Decent product,RamonI,It’s a good antivirus program it does keep my ...,4,0,True
28190,41,BB41,359,Works great,GracieW,"Works great, never had a problem with it, will...",5,0,True


In [10]:
# df_products.to_excel("BB_Product_data_all.xlsx", index=False)
# df_reviews.to_excel("BB_Review_data_all.xlsx", index=False)

#df_review1.to_excel("../data/BB_Review_data.xlsx", index=False)
df_review1.to_csv("../data/BB_Review_data_wID.csv",index=False)

In [15]:
p_df = pd.read_excel('BB_Product_data_all.xlsx')
r_df = pd.read_excel('BB_Review_data_all.xlsx')

In [16]:
p_df.head()

,Name,Star_rating,Total_reviews
0,Apple - iPhone 14 128GB (Unlocked) - Midnight,4.9,209
1,Samsung - Galaxy S22 Ultra 128GB - Phantom Bla...,4.6,396
2,HP - DeskJet 2755e Wireless Inkjet Printer wit...,4.1,3489
3,JBL - Flip 5 Portable Bluetooth Speaker - Black,4.8,5932
4,Microsoft - 365 Family (Up to 6 People) (12-Mo...,4.5,10353


In [17]:
r_df.head()

,Reviewer_Name,Review_Title,Star_Rating,Review_Content,Helpfulness_votes
0,AmandaD,Awesome experience all around,5,"Husband upgraded from a 12 to a 14, loves it. ...",0
1,MauraC,Great phone,5,It’s a pretty great phone. It has all that I n...,0
2,Peachy,It’s an iPhone,5,"It’s a unlocked iPhone, I was able to pick up ...",0
3,PauletteP,Hello….Can You Hear Me Now?,5,Apple products are very good products but of c...,0
4,AnthonyC,Great!!,5,Awesome product love it so much. I would recom...,0


In [33]:
prods = pd.read_excel("../data/BB_products.xlsx")

In [34]:
prods.head()

,productID,prodSiteID,site,URL,product_title,product_details,product_specs,total_star_rating,total_reviews,product_price
0,1,BB1,bestbuy,https://www.bestbuy.com/site/apple-iphone-14-1...,Apple - iPhone 14 128GB (Unlocked) - Midnight,iPhone 14. With the most impressive dual-camer...,Key Specs\nScreen Size\n6.1 inches\nFront-Faci...,4.9,211,729.99
1,2,BB2,bestbuy,https://www.bestbuy.com/site/samsung-galaxy-s2...,Samsung - Galaxy S22 Ultra 128GB - Phantom Bla...,"The power of our fastest chip ever, long lasti...",Key Specs Screen Size 6.8 inches\nFront-Facing...,4.6,396,1199.99
2,3,BB3,bestbuy,https://www.bestbuy.com/site/hp-deskjet-2755e-...,HP - DeskJet 2755e Wireless Inkjet Printer wit...,The HP DeskJet 2755e has the essential feature...,Key Specs\nMobile Device Printing\nYes\nNetwor...,4.1,3496,49.99
3,4,BB4,bestbuy,https://www.bestbuy.com/site/jbl-flip-5-portab...,JBL - Flip 5 Portable Bluetooth Speaker - Black,Water-resistant design\nMeets IPX7 standards.\...,Wireless\nYes\nConnectivity Technology\nBlueto...,4.8,5938,89.99
4,5,BB5,bestbuy,https://www.bestbuy.com/site/microsoft-365-fam...,Microsoft - 365 Family (Up to 6 People) (12-Mo...,Premium Office apps included\nGet premium vers...,Key Specs\nNumber of Users\n6\nNumber of Devic...,4.5,10362,99.99


In [35]:
test_blob = textblob.TextBlob(prods.loc[0]['product_details'])

In [36]:
prods['description_subjectivity'] = None

In [40]:
stop = set(stopwords.words("english"))
for i,row in prods.iterrows():
    blob = textblob.TextBlob(row['product_details'] + "\n" + row['product_specs'])

    subj = blob.subjectivity

    row['description_subjectivity'] = float(subj)
    prods.loc[i] = row


#prods['product_price'] = prods['product_price'].str.replace('\$|,','',regex=True)

prods.head()

,productID,prodSiteID,site,URL,product_title,product_details,product_specs,total_star_rating,total_reviews,product_price,description_subjectivity
0,1,BB1,bestbuy,https://www.bestbuy.com/site/apple-iphone-14-1...,Apple - iPhone 14 128GB (Unlocked) - Midnight,iPhone 14. With the most impressive dual-camer...,Key Specs\nScreen Size\n6.1 inches\nFront-Faci...,4.9,211,729.99,0.437121
1,2,BB2,bestbuy,https://www.bestbuy.com/site/samsung-galaxy-s2...,Samsung - Galaxy S22 Ultra 128GB - Phantom Bla...,"The power of our fastest chip ever, long lasti...",Key Specs Screen Size 6.8 inches\nFront-Facing...,4.6,396,1199.99,0.48015
2,3,BB3,bestbuy,https://www.bestbuy.com/site/hp-deskjet-2755e-...,HP - DeskJet 2755e Wireless Inkjet Printer wit...,The HP DeskJet 2755e has the essential feature...,Key Specs\nMobile Device Printing\nYes\nNetwor...,4.1,3496,49.99,0.480135
3,4,BB4,bestbuy,https://www.bestbuy.com/site/jbl-flip-5-portab...,JBL - Flip 5 Portable Bluetooth Speaker - Black,Water-resistant design\nMeets IPX7 standards.\...,Wireless\nYes\nConnectivity Technology\nBlueto...,4.8,5938,89.99,0.35085
4,5,BB5,bestbuy,https://www.bestbuy.com/site/microsoft-365-fam...,Microsoft - 365 Family (Up to 6 People) (12-Mo...,Premium Office apps included\nGet premium vers...,Key Specs\nNumber of Users\n6\nNumber of Devic...,4.5,10362,99.99,0.460182


In [41]:
prods.info()
prods['description_subjectivity'] = prods['description_subjectivity'].astype(float)
prods.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   productID                 15 non-null     int64  
 1   prodSiteID                15 non-null     object 
 2   site                      15 non-null     object 
 3   URL                       15 non-null     object 
 4   product_title             15 non-null     object 
 5   product_details           15 non-null     object 
 6   product_specs             15 non-null     object 
 7   total_star_rating         15 non-null     float64
 8   total_reviews             15 non-null     int64  
 9   product_price             15 non-null     float64
 10  description_subjectivity  15 non-null     object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.4+ KB


,count,mean,std,min,25%,50%,75%,max
productID,15.0,8.000000,4.472136,1.00000,4.500000,8.000000,11.500000,15.000000
total_star_rating,15.0,4.580000,0.256905,4.00000,4.500000,4.600000,4.800000,4.900000
total_reviews,15.0,1742.400000,2875.192659,28.00000,205.500000,396.000000,1513.000000,10362.000000
product_price,15.0,352.990000,472.209094,34.99000,59.990000,99.990000,374.990000,1599.990000
description_subjectivity,15.0,0.449737,0.057999,0.35085,0.430166,0.437013,0.470158,0.622115


In [42]:
prods[['review_subjectivity','review_polarity']] = None,None
for i,row in prods.iterrows():
    #include the header in the summary
    #s = row['review_header'] + row['review_content']
    blob = textblob.TextBlob(row['review_content'])
    row[['review_subjectivity','review_polarity']] = float(blob.subjectivity),float(blob.polarity)
    prods.loc[i] = row



KeyError: 'review_content'